# Tratamento dos dados para camada Silver.

## Criaçao de um ambiente spark para tratar os dados.
## Posteriormente irei transformar os codigos para pandas e ou numpy e inseri-los no projeto.

In [73]:
%%bash

# Instal Java
apt-get update --fix-missing
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (130 kB/s)
Reading package lists...


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [74]:
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when


os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [75]:
path = "/content/bs_bronze.parquet"

In [98]:
df_br_data = spark.read.format("parquet").load(path)

In [99]:
df_br_data.show(5, False)


+------------------------------------+-----------------------+------------+---------------------+---------+---------+--------------+--------------+-----------+-------------+------------------+-----------------+----------+----------------------------+----------+---------------------+
|id                                  |name                   |brewery_type|address_1            |address_2|address_3|city          |state_province|postal_code|country      |longitude         |latitude         |phone     |website_url                 |state     |street               |
+------------------------------------+-----------------------+------------+---------------------+---------+---------+--------------+--------------+-----------+-------------+------------------+-----------------+----------+----------------------------+----------+---------------------+
|5128df48-79fc-4f0f-8b52-d06be54d0cec|(405) Brewing Co       |micro       |1716 Topeka St       |NULL     |NULL     |Norman        |Oklahoma      |7

In [100]:
df_br_data.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- brewery_type: string (nullable = true)
 |-- address_1: string (nullable = true)
 |-- address_2: string (nullable = true)
 |-- address_3: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- website_url: string (nullable = true)
 |-- state: string (nullable = true)
 |-- street: string (nullable = true)



Exploração dos campos de localização, pois alguns veiram faltando.

In [101]:
df_br_data.select("country").distinct().show()

+-------------+
|      country|
+-------------+
|United States|
|      Ireland|
+-------------+



In [102]:
# Aqui eu verifiquei que existem 3 id sem informação de endeço, em nenhum lugar, e vi que mais um id, possui algo estranho, como se não tivesse nome de rua,
# vou investigar melhor depois que organizar os tres id faltantes.


df_br_data.select("id", "address_1", "address_2", "address_3", "longitude", "latitude").orderBy(col("address_1").asc()).show(50, False)

+------------------------------------+------------------------------+---------+---------+------------------+-----------------+
|id                                  |address_1                     |address_2|address_3|longitude         |latitude         |
+------------------------------------+------------------------------+---------+---------+------------------+-----------------+
|0faa0fb2-fffa-416d-9eab-46f67477c8ef|NULL                          |NULL     |NULL     |-111.5860662      |33.436188        |
|b7b68d22-5045-4501-b9bf-ec94946eaffc|NULL                          |NULL     |NULL     |NULL              |NULL             |
|fe6b9893-b93e-43d5-a9f6-3e0c89a3f13c|NULL                          |NULL     |NULL     |-119.9036592      |37.570148        |
|46839a79-b7bf-4733-b91b-ce116d062a57|100 B South Olive St          |NULL     |NULL     |-88.750264        |34.001703        |
|4ffda196-dd59-44a5-9eeb-5f7fd4b58f5a|1043 Park St                  |NULL     |NULL     |-104.8667206      |39.

In [103]:
#df_br_data.select("address_1").distinct().orderBy(col("address_1").asc()).show(50, False)
#df_br_data.select("address_2").distinct().orderBy(col("address_1").asc()).show(50, False)
df_br_data.select("address_3").distinct().orderBy(col("address_1").asc()).show(50, False)

+---------+
|address_3|
+---------+
|NULL     |
+---------+



In [104]:
df_dup_add1 = (df_br_data.select("id", "address_1", "address_2", "address_3")
                .groupBy(["address_1", "address_2", "address_3"])
                    .count()
                        .where('count > 1')
                        .sort('count', ascending=False)
               )


In [83]:
df_dup_add1.show()

+---------+---------+---------+-----+
|address_1|address_2|address_3|count|
+---------+---------+---------+-----+
|     NULL|     NULL|     NULL|    3|
+---------+---------+---------+-----+



In [105]:
df_null_add = (df_br_data.select("*")
                .filter(col("id").isin("0faa0fb2-fffa-416d-9eab-46f67477c8ef", "b7b68d22-5045-4501-b9bf-ec94946eaffc", "fe6b9893-b93e-43d5-a9f6-3e0c89a3f13c"))
               )

In [106]:
df_null_add.show(3, False)

+------------------------------------+---------------------------------------------+------------+---------+---------+---------+---------+--------------+-----------+-------------+------------+---------+----------+-----------------------------+----------+------+
|id                                  |name                                         |brewery_type|address_1|address_2|address_3|city     |state_province|postal_code|country      |longitude   |latitude |phone     |website_url                  |state     |street|
+------------------------------------+---------------------------------------------+------------+---------+---------+---------+---------+--------------+-----------+-------------+------------+---------+----------+-----------------------------+----------+------+
|0faa0fb2-fffa-416d-9eab-46f67477c8ef|12 West Brewing Company - Production Facility|micro       |NULL     |NULL     |NULL     |Mesa     |Arizona       |85207      |United States|-111.5860662|33.436188|NULL      |NULL 

In [107]:
df_att = df_br_data.withColumn("address_1", when(col("id") == "0faa0fb2-fffa-416d-9eab-46f67477c8ef", "12 W Main St").otherwise(col("address_1")))

### Alteraçao do campo "address_1" nas lojas faltantes.

In [108]:
df_br_att = df_br_data.withColumn(
    "address_1",
    when(col("id") == "0faa0fb2-fffa-416d-9eab-46f67477c8ef", "12 W Main St")
    .when(col("id") == "b7b68d22-5045-4501-b9bf-ec94946eaffc", "36846 Co Hwy 66")
    .when(col("id") == "fe6b9893-b93e-43d5-a9f6-3e0c89a3f13c", "5114 Yosemite All-Year Hwy")
    .otherwise(col("address_1"))
)

In [109]:
df_br_att.select("*").filter(col("id").isin("0faa0fb2-fffa-416d-9eab-46f67477c8ef", "b7b68d22-5045-4501-b9bf-ec94946eaffc", "fe6b9893-b93e-43d5-a9f6-3e0c89a3f13c"))


id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0faa0fb2-fffa-416...,12 West Brewing C...,micro,12 W Main St,NULL,NULL,Mesa,Arizona,85207,United States,-111.5860662,33.436188,NULL,NULL,Arizona,NULL
b7b68d22-5045-450...,14 Lakes Brewery,micro,36846 Co Hwy 66,NULL,NULL,Crosslake,Minnesota,56442,United States,NULL,NULL,2186924129,NULL,Minnesota,NULL
fe6b9893-b93e-43d...,1850 Brewing Company,micro,5114 Yosemite All...,NULL,NULL,Mariposa,California,95338,United States,-119.9036592,37.570148,NULL,http://www.1850re...,California,NULL


### Alteraçao do campo "longitude e latitude" nas lojas faltantes.

In [110]:
df_br_att.select("id", "address_1", "address_2", "address_3", "latitude", "longitude").orderBy(col("address_1").asc()).show(50, False)

+------------------------------------+------------------------------+---------+---------+-----------------+------------------+
|id                                  |address_1                     |address_2|address_3|latitude         |longitude         |
+------------------------------------+------------------------------+---------+---------+-----------------+------------------+
|46839a79-b7bf-4733-b91b-ce116d062a57|100 B South Olive St          |NULL     |NULL     |34.001703        |-88.750264        |
|4ffda196-dd59-44a5-9eeb-5f7fd4b58f5a|1043 Park St                  |NULL     |NULL     |39.38269495      |-104.8667206      |
|84d621c4-81a5-44e6-aca7-1566c2e67cc0|11109 Plank Rd                |NULL     |NULL     |NULL             |NULL              |
|f41a0c47-ba9b-4547-bfed-fcbefe0fc74b|1129 Irvin Garrish Hwy        |NULL     |NULL     |35.10715368      |-75.97176063      |
|e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2|1135 NW Galveston Ave Ste B   |NULL     |NULL     |44.0575649       |-121

In [118]:
df_br_att.select("*").filter(col("id").isin("b7b68d22-5045-4501-b9bf-ec94946eaffc", "84d621c4-81a5-44e6-aca7-1566c2e67cc0",
                                            "d35b40b0-a3ff-4878-a6ee-9caa2149b521", "5c53b314-ebab-4e3e-89be-e4139d9318ae",
                                            "50521ef7-f543-4c5d-98b1-0d0ee1a2be01", "45119c56-345b-4adc-b481-c5cf7bfe98c4",
                                            "fb94830f-6196-4f59-9189-c9060b778085", "9c5a66c8-cc13-416f-a5d9-0a769c87d318",
                                            "9c5a66c8-cc13-416f-a5d9-0a769c87d318", "d81ff708-b5d2-478f-af6a-6d40f5beb9ac",
                                            "6c53984f-fac1-4ea7-9c44-44e25897c71a")).show(11, False)


+------------------------------------+------------------------+------------+------------------------------+---------+---------+-------------+--------------+-----------+-------------+------------+----------+----------+-------------------------------+--------------+------------------------------+
|id                                  |name                    |brewery_type|address_1                     |address_2|address_3|city         |state_province|postal_code|country      |longitude   |latitude  |phone     |website_url                    |state         |street                        |
+------------------------------------+------------------------+------------+------------------------------+---------+---------+-------------+--------------+-----------+-------------+------------+----------+----------+-------------------------------+--------------+------------------------------+
|9c5a66c8-cc13-416f-a5d9-0a769c87d318|(512) Brewing Co        |micro       |407 Radam Ln Ste F200         |NULL 

In [119]:
#Alteraçao dos endereços "address_1" nas lojas faltantes.
df_br_att = df_br_att.withColumn(
    "latitude",
     when(col("id") == "9c5a66c8-cc13-416f-a5d9-0a769c87d318", "30.2232157")
    .when(col("id") == "d81ff708-b5d2-478f-af6a-6d40f5beb9ac", "42.9539343")
    .when(col("id") == "fb94830f-6196-4f59-9189-c9060b778085", "33.3219732")
    .when(col("id") == "50521ef7-f543-4c5d-98b1-0d0ee1a2be01", "44.9477074")
    .when(col("id") == "45119c56-345b-4adc-b481-c5cf7bfe98c4", "34.9206198")
    .when(col("id") == "5c53b314-ebab-4e3e-89be-e4139d9318ae", "35.8009165")
    .when(col("id") == "b7b68d22-5045-4501-b9bf-ec94946eaffc", "46.6876583")
    .when(col("id") == "d35b40b0-a3ff-4878-a6ee-9caa2149b521", "39.4171642")
    .when(col("id") == "84d621c4-81a5-44e6-aca7-1566c2e67cc0", "38.3279312")
    .when(col("id") == "6c53984f-fac1-4ea7-9c44-44e25897c71a", "44.8149712")
    .otherwise(col("latitude"))
)

In [120]:
df_br_att.select("*").filter(col("id").isin("b7b68d22-5045-4501-b9bf-ec94946eaffc", "84d621c4-81a5-44e6-aca7-1566c2e67cc0",
                                            "d35b40b0-a3ff-4878-a6ee-9caa2149b521", "5c53b314-ebab-4e3e-89be-e4139d9318ae",
                                            "50521ef7-f543-4c5d-98b1-0d0ee1a2be01", "45119c56-345b-4adc-b481-c5cf7bfe98c4",
                                            "fb94830f-6196-4f59-9189-c9060b778085", "9c5a66c8-cc13-416f-a5d9-0a769c87d318",
                                            "9c5a66c8-cc13-416f-a5d9-0a769c87d318", "d81ff708-b5d2-478f-af6a-6d40f5beb9ac",
                                            "6c53984f-fac1-4ea7-9c44-44e25897c71a")).show(11, False)

+------------------------------------+------------------------+------------+------------------------------+---------+---------+-------------+--------------+-----------+-------------+------------+----------+----------+-------------------------------+--------------+------------------------------+
|id                                  |name                    |brewery_type|address_1                     |address_2|address_3|city         |state_province|postal_code|country      |longitude   |latitude  |phone     |website_url                    |state         |street                        |
+------------------------------------+------------------------+------------+------------------------------+---------+---------+-------------+--------------+-----------+-------------+------------+----------+----------+-------------------------------+--------------+------------------------------+
|9c5a66c8-cc13-416f-a5d9-0a769c87d318|(512) Brewing Co        |micro       |407 Radam Ln Ste F200         |NULL 

In [121]:
df_br_att = df_br_att.withColumn(
    "longitude",
     when(col("id") == "9c5a66c8-cc13-416f-a5d9-0a769c87d318", "-97.7701612")
    .when(col("id") == "d81ff708-b5d2-478f-af6a-6d40f5beb9ac", "-78.7127541")
    .when(col("id") == "fb94830f-6196-4f59-9189-c9060b778085", "-111.7267596")
    .when(col("id") == "50521ef7-f543-4c5d-98b1-0d0ee1a2be01", "-93.0895051")
    .when(col("id") == "45119c56-345b-4adc-b481-c5cf7bfe98c4", "-82.2846578")
    .when(col("id") == "5c53b314-ebab-4e3e-89be-e4139d9318ae", "-78.6345215")
    .when(col("id") == "b7b68d22-5045-4501-b9bf-ec94946eaffc", "-94.1123597")
    .when(col("id") == "d35b40b0-a3ff-4878-a6ee-9caa2149b521", "-76.9773789")
    .when(col("id") == "84d621c4-81a5-44e6-aca7-1566c2e67cc0", "-77.7187148")
    .when(col("id") == "6c53984f-fac1-4ea7-9c44-44e25897c71a", "-73.0816367")
    .otherwise(col("longitude"))
)

In [122]:
df_br_att.select("*").filter(col("id").isin("b7b68d22-5045-4501-b9bf-ec94946eaffc", "84d621c4-81a5-44e6-aca7-1566c2e67cc0",
                                            "d35b40b0-a3ff-4878-a6ee-9caa2149b521", "5c53b314-ebab-4e3e-89be-e4139d9318ae",
                                            "50521ef7-f543-4c5d-98b1-0d0ee1a2be01", "45119c56-345b-4adc-b481-c5cf7bfe98c4",
                                            "fb94830f-6196-4f59-9189-c9060b778085", "9c5a66c8-cc13-416f-a5d9-0a769c87d318",
                                            "9c5a66c8-cc13-416f-a5d9-0a769c87d318", "d81ff708-b5d2-478f-af6a-6d40f5beb9ac",
                                            "6c53984f-fac1-4ea7-9c44-44e25897c71a")).show(11, False)

+------------------------------------+------------------------+------------+------------------------------+---------+---------+-------------+--------------+-----------+-------------+------------+----------+----------+-------------------------------+--------------+------------------------------+
|id                                  |name                    |brewery_type|address_1                     |address_2|address_3|city         |state_province|postal_code|country      |longitude   |latitude  |phone     |website_url                    |state         |street                        |
+------------------------------------+------------------------+------------+------------------------------+---------+---------+-------------+--------------+-----------+-------------+------------+----------+----------+-------------------------------+--------------+------------------------------+
|9c5a66c8-cc13-416f-a5d9-0a769c87d318|(512) Brewing Co        |micro       |407 Radam Ln Ste F200         |NULL 

In [126]:
df_br_att.select("*").filter(col("id").isin("34e8c68b-6146-453f-a4b9-1f6cd99a5ada")).show(1, False)

+------------------------------------+-----------------------+------------+-------------------+---------+---------+--------------+--------------+-----------+-------------+-----------+-----------------+----------+----------------------------+---------+-------------------+
|id                                  |name                   |brewery_type|address_1          |address_2|address_3|city          |state_province|postal_code|country      |longitude  |latitude         |phone     |website_url                 |state    |street             |
+------------------------------------+-----------------------+------------+-------------------+---------+---------+--------------+--------------+-----------+-------------+-----------+-----------------+----------+----------------------------+---------+-------------------+
|34e8c68b-6146-453f-a4b9-1f6cd99a5ada|1 of Us Brewing Company|micro       |8100 Washington Ave|NULL     |NULL     |Mount Pleasant|Wisconsin     |53406-3920 |United States|-87.8833635|4

In [127]:
#df_br_att = df_br_att.withColumn(
#    "longitude",
#     when(col("id") == "34e8c68b-6146-453f-a4b9-1f6cd99a5ada", "-87.8833635")
#    .otherwise(col("longitude"))
#)

df_br_att = df_br_att.withColumn(
    "latitude",
     when(col("id") == "34e8c68b-6146-453f-a4b9-1f6cd99a5ada", "42.7201082")
    .otherwise(col("latitude"))
)

In [128]:
df_br_att.select("id", "address_1", "address_2", "address_3", "latitude", "longitude").orderBy(col("address_1").asc()).show(50, False)

+------------------------------------+------------------------------+---------+---------+-----------+------------+
|id                                  |address_1                     |address_2|address_3|latitude   |longitude   |
+------------------------------------+------------------------------+---------+---------+-----------+------------+
|46839a79-b7bf-4733-b91b-ce116d062a57|100 B South Olive St          |NULL     |NULL     |34.001703  |-88.750264  |
|4ffda196-dd59-44a5-9eeb-5f7fd4b58f5a|1043 Park St                  |NULL     |NULL     |39.38269495|-104.8667206|
|84d621c4-81a5-44e6-aca7-1566c2e67cc0|11109 Plank Rd                |NULL     |NULL     |38.3279312 |-77.7187148 |
|f41a0c47-ba9b-4547-bfed-fcbefe0fc74b|1129 Irvin Garrish Hwy        |NULL     |NULL     |35.10715368|-75.97176063|
|e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2|1135 NW Galveston Ave Ste B   |NULL     |NULL     |44.0575649 |-121.3288021|
|d35b40b0-a3ff-4878-a6ee-9caa2149b521|1146 colonel Joshua Ct        |NULL     |N

In [135]:
df_br_att.select("*").filter(col("id").isin("e5f3e72a-fee2-4813-82cf-f2e53b439ae6")).show(1, False)

+------------------------------------+------------------------+------------+---------+---------+---------+---------+--------------+-----------+-------+------------+-----------+------------+--------------------------+-----+--------------+
|id                                  |name                    |brewery_type|address_1|address_2|address_3|city     |state_province|postal_code|country|longitude   |latitude   |phone       |website_url               |state|street        |
+------------------------------------+------------------------+------------+---------+---------+---------+---------+--------------+-----------+-------+------------+-----------+------------+--------------------------+-----+--------------+
|e5f3e72a-fee2-4813-82cf-f2e53b439ae6|12 Acres Brewing Company|micro       |Clonmore |NULL     |NULL     |Killeshin|Laois         |R93 X3X8   |Ireland|-6.979343891|52.84930763|353599107299|https://12acresbrewing.ie/|Laois|Unnamed Street|
+------------------------------------+----------

In [134]:
#df_br_att = df_br_att.withColumn(
#    "address_1",
#     when(col("id") == "e5f3e72a-fee2-4813-82cf-f2e53b439ae6", "Clonmore")
#    .otherwise(col("address_1"))
#)

df_br_att = df_br_att.withColumn(
    "address_2",
     when(col("id") == "e5f3e72a-fee2-4813-82cf-f2e53b439ae6", "NULL")
    .otherwise(col("address_2"))
)

In [136]:
df_br_att.select("id", "address_1", "address_2", "address_3", "latitude", "longitude").orderBy(col("address_1").asc()).show(50, False)

+------------------------------------+------------------------------+---------+---------+-----------+------------+
|id                                  |address_1                     |address_2|address_3|latitude   |longitude   |
+------------------------------------+------------------------------+---------+---------+-----------+------------+
|46839a79-b7bf-4733-b91b-ce116d062a57|100 B South Olive St          |NULL     |NULL     |34.001703  |-88.750264  |
|4ffda196-dd59-44a5-9eeb-5f7fd4b58f5a|1043 Park St                  |NULL     |NULL     |39.38269495|-104.8667206|
|84d621c4-81a5-44e6-aca7-1566c2e67cc0|11109 Plank Rd                |NULL     |NULL     |38.3279312 |-77.7187148 |
|f41a0c47-ba9b-4547-bfed-fcbefe0fc74b|1129 Irvin Garrish Hwy        |NULL     |NULL     |35.10715368|-75.97176063|
|e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2|1135 NW Galveston Ave Ste B   |NULL     |NULL     |44.0575649 |-121.3288021|
|d35b40b0-a3ff-4878-a6ee-9caa2149b521|1146 colonel Joshua Ct        |NULL     |N

In [138]:
df_br_att.show(50, False)

+------------------------------------+---------------------------------------------+------------+------------------------------+---------+---------+----------------+--------------+-----------+-------------+------------+-----------+------------+----------------------------------+--------------+------------------------------+
|id                                  |name                                         |brewery_type|address_1                     |address_2|address_3|city            |state_province|postal_code|country      |longitude   |latitude   |phone       |website_url                       |state         |street                        |
+------------------------------------+---------------------------------------------+------------+------------------------------+---------+---------+----------------+--------------+-----------+-------------+------------+-----------+------------+----------------------------------+--------------+------------------------------+
|5128df48-79fc-4f0f-8b